# ML Signal-Based Credit Score

Converts ML model outputs into **credit score** (300-900 range)

In [1]:
import pandas as pd
import numpy as np
from dataclasses import dataclass
import matplotlib.pyplot as plt
import seaborn as sns

### ML Features Data Structure

In [2]:
@dataclass
class MLFeatures:
    """ML model outputs for sector scoring"""
    PD: float                    # Probability of Default (0-1)
    anomaly_score: float         # Isolation Forest score
    prob_high: float             # High risk probability (0-1)
    prob_medium: float           # Medium risk probability (0-1)
    prob_low: float              # Low risk probability (0-1)

### Sector Scoring Functions

> ### Convert ML outputs to risk sectors (0-6)

In [3]:
def get_pd_sector(pd: float) -> int:
    """Sector A: PD (0-6)"""
    if pd <= 0.05:
        return 0
    elif pd <= 0.10:
        return 1
    elif pd <= 0.20:
        return 2
    elif pd <= 0.35:
        return 3
    elif pd <= 0.50:
        return 4
    elif pd <= 0.70:
        return 5
    else:
        return 6

def get_anomaly_sector(anomaly_score: float) -> int:
    """Sector B: Anomaly Score (0-6)"""
    if anomaly_score >= 0.15:
        return 0
    elif anomaly_score >= 0.05:
        return 1
    elif anomaly_score >= 0.00:
        return 2
    elif anomaly_score >= -0.05:
        return 3
    elif anomaly_score >= -0.10:
        return 4
    elif anomaly_score >= -0.20:
        return 5
    else:
        return 6

def get_prob_high_sector(prob_high: float) -> int:
    """Sector C: prob_high (0-6)"""
    if prob_high <= 0.05:
        return 0
    elif prob_high <= 0.10:
        return 1
    elif prob_high <= 0.20:
        return 2
    elif prob_high <= 0.35:
        return 3
    elif prob_high <= 0.50:
        return 4
    elif prob_high <= 0.70:
        return 5
    else:
        return 6

def get_prob_medium_sector(prob_medium: float) -> int:
    """Sector D: prob_medium (0-6)"""
    if prob_medium <= 0.10:
        return 0
    elif prob_medium <= 0.20:
        return 1
    elif prob_medium <= 0.30:
        return 2
    elif prob_medium <= 0.45:
        return 3
    elif prob_medium <= 0.60:
        return 4
    elif prob_medium <= 0.75:
        return 5
    else:
        return 6

def get_prob_low_sector(prob_low: float) -> int:
    """Sector E: prob_low (0-6) - INVERSE logic"""
    if prob_low >= 0.80:
        return 0
    elif prob_low >= 0.65:
        return 1
    elif prob_low >= 0.50:
        return 2
    elif prob_low >= 0.35:
        return 3
    elif prob_low >= 0.20:
        return 4
    elif prob_low >= 0.10:
        return 5
    else:
        return 6

In [4]:
def calculate_ml_sector_score(ml_features: MLFeatures) -> int:
    
    BASE_SCORE = 500
    
    # Sector weights
    WEIGHT_PD = 1.5
    WEIGHT_ANOMALY = 1.2
    WEIGHT_PROB_HIGH = 1.3
    WEIGHT_PROB_MEDIUM = 0.8
    WEIGHT_PROB_LOW = 1.1
    
    # Base penalties per sector level
    PENALTY_PD = 15
    PENALTY_ANOMALY = 10
    PENALTY_PROB_HIGH = 12.5
    PENALTY_PROB_MEDIUM = 8
    PENALTY_PROB_LOW = 11
    
    # Get sectors
    sector_a = get_pd_sector(ml_features.PD)
    sector_b = get_anomaly_sector(ml_features.anomaly_score)
    sector_c = get_prob_high_sector(ml_features.prob_high)
    sector_d = get_prob_medium_sector(ml_features.prob_medium)
    sector_e = get_prob_low_sector(ml_features.prob_low)
    
    # Calculate penalties
    penalty_a = sector_a * PENALTY_PD * WEIGHT_PD
    penalty_b = sector_b * PENALTY_ANOMALY * WEIGHT_ANOMALY
    penalty_c = sector_c * PENALTY_PROB_HIGH * WEIGHT_PROB_HIGH
    penalty_d = sector_d * PENALTY_PROB_MEDIUM * WEIGHT_PROB_MEDIUM
    penalty_e = sector_e * PENALTY_PROB_LOW * WEIGHT_PROB_LOW
    
    total_penalty = penalty_a + penalty_b + penalty_c + penalty_d + penalty_e
    
    # Calculate final score
    ml_score = BASE_SCORE - total_penalty
    
    # Clamp to valid range
    return int(max(300, min(900, ml_score)))

## 🎲 Noise Functions for ML Training

Add realistic variation to ML scores for training

### Normal Distribution (Gaussian Noise)

```
    σ = Standard Deviation (Controls spread)
    
      High Score (750+)  →  Small σ (10)  = Very Stable
      Medium Score (600-750) →  Large σ (25)  = More Uncertain  
      Low Score (<600)   →  Medium σ (15) = Risky but Bounded
```

In [5]:
def add_industry_noise(ml_score: int) -> int:
    
    # Adaptive standard deviation based on score tier
    if ml_score >= 750:
        sigma = 10   # Very stable - high quality
    elif ml_score >= 600:
        sigma = 25   # Medium uncertainty - borderline cases
    else:
        sigma = 15   # Risky but bounded - low credit
    
    # Generate Gaussian noise
    noise = np.random.normal(loc=0, scale=sigma)
    noisy_score = ml_score + noise
    
    # Clamp to valid range
    return int(np.clip(noisy_score, 300, 900))

In [6]:
def feature_aware_noise(ml_score: int, pd: float, anomaly_flag: int) -> int:
    base_sigma = 20
    
    # Increase uncertainty for risky signals
    if pd > 0.45:
        base_sigma += 10  # High default probability = unpredictable
    
    if anomaly_flag == 1:
        base_sigma += 15  # Anomalous behavior = instability
    
    noise = np.random.normal(0, base_sigma)
    return int(np.clip(ml_score + noise, 300, 900))

### 🏭 Production Function

In [7]:
def calculate_noisy_ml_score(ml_features: MLFeatures) -> int:
    # Step 1: Calculate base ML sector score
    ml_score = calculate_ml_sector_score(ml_features)
    
    # Step 2: Apply both noise functions
    # First apply tier-based noise
    noisy_score = add_industry_noise(ml_score)
    
    # Then apply feature-aware noise
    anomaly_flag = 1 if ml_features.anomaly_score < -0.05 else 0
    noisy_score = feature_aware_noise(
        noisy_score,
        ml_features.PD,
        anomaly_flag
    )
    
    return noisy_score

---
## 🏭 ML-Signal Based Credit Score

**Input CSVs:**
- `feature_with_pd_prediction.csv` – PD model outputs  
- `feature_withanomaly.csv` – Anomaly detection outputs  
- `feature_with_risk_predictions.csv` – Risk classification outputs  

**Output CSV:**
- `feature_with_ml_signal_score.csv` – Feature with ML Signal Credit Score


### 🔁 Process Flow

<div align="center">

| PD   | anomaly_score | prob_high | prob_medium | prob_low |
|------|---------------|-----------|-------------|----------|
| 0.18 | 0.08          | 0.12      | 0.25        | 0.63     |

</div>

&nbsp;

<div align="center">

**▼**

**ML Signal Credit Score Engine**

**▼**

</div>

&nbsp;

<div align="center">

| ... | mlSignalScore |
|-----|---------------|
| ... | 373           |

</div>


In [8]:
# Load all ML prediction CSVs
df_base = pd.read_csv('../../../3. Data/1. Raw_Features/features_only.csv')
df_pd = pd.read_csv('../../../3. Data/2. PD_Data/features_with_pd_predictions.csv')
df_anomaly = pd.read_csv('../../../3. Data/3. Anomaly_Data/features_with_anomaly.csv')
df_risk = pd.read_csv('../../../3. Data/4. Risk_Data/features_with_risk_predictions.csv')

# Merge ML features
df = df_base.copy()
df['PD'] = df_pd['PD']
df['anomaly_score'] = df_anomaly['anomaly_score']
df['prob_high'] = df_risk['prob_high']
df['prob_medium'] = df_risk['prob_medium']
df['prob_low'] = df_risk['prob_low']

# Calculate ML signal scores for all rows
ml_scores = []

for idx, row in df.iterrows():
    ml_features = MLFeatures(
        PD=row['PD'],
        anomaly_score=row['anomaly_score'],
        prob_high=row['prob_high'],
        prob_medium=row['prob_medium'],
        prob_low=row['prob_low']
    )

    score = calculate_noisy_ml_score(ml_features)
    ml_scores.append(score)

# Add to dataframe and save
df['MLSignalBasedCreditScore'] = ml_scores
df.to_csv('../../../3. Data/5*. Hybrid_Data/feature_with_ml_signal_score.csv', index=False)

print("Saved ML signal based credit scores to feature_with_ml_signal_score.csv")

Saved ML signal based credit scores to feature_with_ml_signal_score.csv
